In [1]:
import os
import time
import math
import json
import redis
import random
import asyncio
import traceback
from datetime import datetime
from aredis import StrictRedis

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.utils import Utils
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

In [4]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [5]:
Utils.update_symbols()
symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1
rd.set(f'hq_assist_count', assist_count)

True

In [6]:
date = time.strftime('%Y%m%d')
date = '20210712'
date

'20210712'

In [7]:
dd = DailyData.load(dt=date)

In [129]:
dd.close_sharedmemory()
dd.unlink_sharedmemory()

In [8]:
async def start_fake_data_generator(assist_count=7, snapshot_interval=5):
    assist_indices = list([ _ for _ in range(assist_count)])
    
    for _, check_point in enumerate(dd.check_points):
        random.shuffle(assist_indices)
        try:
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx]
            compute_stats(dd.snapshots[_], dd.basics, dd.statistic[_], fs5p, time_lapse)

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps({"status":'successful',"date":date,"idx":_,'check_point':int(check_point)})
                )
                
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'sussess', end='\r')

        except Exception as e:
            error = {
                "status": 'failed',
                "idx": _,
                'check_point':int(check_point),
                "exception": str(e),
                'traceback': traceback.format_exc()
            }

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps(error)
                )
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'failed', str(e), end='\r')
        finally:
            if _ > 115:
                await asyncio.sleep(snapshot_interval)


In [11]:
task = asyncio.create_task(start_fake_data_generator())

In [10]:
task.cancel()

True

In [17]:
import numpy as np

In [20]:
np.array([[1],[2],[3],[4]]).flatten().tolist()

[1, 2, 3, 4]

In [13]:
json.dumps(dd.check_points.tolist())

'[1626052490, 1626052500, 1626052505, 1626052510, 1626052515, 1626052520, 1626052525, 1626052530, 1626052535, 1626052540, 1626052545, 1626052550, 1626052555, 1626052560, 1626052565, 1626052570, 1626052575, 1626052580, 1626052585, 1626052590, 1626052595, 1626052600, 1626052605, 1626052610, 1626052615, 1626052620, 1626052625, 1626052630, 1626052635, 1626052640, 1626052645, 1626052650, 1626052655, 1626052660, 1626052665, 1626052670, 1626052675, 1626052680, 1626052685, 1626052690, 1626052695, 1626052700, 1626052705, 1626052710, 1626052715, 1626052720, 1626052725, 1626052730, 1626052735, 1626052740, 1626052745, 1626052750, 1626052755, 1626052760, 1626052765, 1626052770, 1626052775, 1626052780, 1626052785, 1626052790, 1626052795, 1626052800, 1626052805, 1626052810, 1626052815, 1626052820, 1626052825, 1626052830, 1626052835, 1626052840, 1626052845, 1626052850, 1626052855, 1626052860, 1626052865, 1626052870, 1626052875, 1626052880, 1626052885, 1626052890, 1626052895, 1626052900, 1626052905, 16

In [ ]:
for key in rd.keys():
    print(key)

In [ ]:
int(rd.get(f'hq_{date}_check_points_length'))

In [ ]:
rd.get(f'hq_{date}_check_points_length')

In [120]:
idx = 2661

In [121]:
result = dd.statistic[idx, :, 4] > 0
result

array([False, False,  True, ..., False, False, False])

In [122]:
len(np.argwhere(result))

154

In [123]:
result2 = dd.statistic[idx-1, :, 4] <= 0
result2

array([ True,  True, False, ...,  True,  True,  True])

In [124]:
len(np.argwhere(result2))

4222

In [125]:
np.argwhere(result == result2)

array([[3348]], dtype=int64)

In [126]:
result2[3448]

True

In [119]:
dd.check_points[np.argmax(dd.statistic[:2703, 3348, 4] > 0)]

1626071480

In [ ]:
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(float(ts)))

In [ ]:
dd.statistic[1000,:,4]

In [ ]:
import numpy as np

In [ ]:
[
    [
        idx[0], 
        dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)]
    ] for idx in np.argwhere(dd.statistic[2000,:,4]>0)
]


In [ ]:
start = time.time()
json.dumps(list([
    (
        int(idx[0]), 
        int(dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)])
    ) for idx in np.argwhere(dd.statistic[2103,:,4]>0)
]))
time.time()-start

In [ ]:
delta = 1926
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1626073210+delta))

In [ ]:
dd.check_points[2103]

In [ ]:
1626052500+delta

In [ ]:
dd.check_points[-1]

In [ ]:
import numpy as np

In [ ]:
np.argwhere(dd.statistic[1358, :, 4] > 0).tolist()

In [ ]:
dd.check_points[1358]